# Convolutional Neural Networks

> Author: Diego Vieira

# Notebook Summary

1. Importing Libraries
2. Building the LeNet-5 Architecture
3. Preparing the Data
  * Creating the Dataset
    * Mounting the Drive
    * Setting DataImageGenerator
  * Classifying COVID-19 dataset with a Convolutional Neural Network
4. Setting Callbacks
5. Viewing the Results
6. Implementing Data Augmentation

# 1. Importing Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.regularizers import l2
import numpy as np
import matplotlib.pyplot as plt
import datetime
import timeit
%load_ext tensorboard
!rm -rf ./logs/
%tensorflow_version 2.x
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# 2. Building the LeNet-5 Model

In [ ]:
class LeNet5(Model):
    """
  LeNet-5 Model

  Attributes
  --------------------------------
  conv1: tf.keras.layers
    Convolutional layer of model

  conv2: tf.keras.layers
    Convolutional layer of model

  max_pool: tf.keras.layers
    MaxPooling layer of model

  flatten: tf.keras.layers
    Flatten layer of model

  dense1: tf.keras.layers
  dense2: tf.keras.layers
    Dense layer of model

  dense3: tf.keras.layers
    Output layer of model

  """
    
    def __init__(self, reg=0):
        """
        Initialize the model.
        :param num_classes:     Number of classes to predict from
        """
        super(LeNet5, self).__init__()
        # We will build the various layers composing LeNet-5:
        self.conv1 = Conv2D(6, kernel_size=(5, 5), padding='same', activation='relu', activity_regularizer=l2(reg))
        self.conv2 = Conv2D(16, kernel_size=(5, 5), activation='relu', activity_regularizer=l2(reg))
        self.max_pool = MaxPooling2D(pool_size=(2, 2))
        self.flatten = Flatten()
        self.dense1 = Dense(120, activation='relu', activity_regularizer=l2(reg))
        self.dense2 = Dense(84, activation='relu', activity_regularizer=l2(reg))
        self.dense3 = Dense(1, activation='sigmoid', activity_regularizer=l2(reg))
        
    def call(self, inputs):
        """
        Call the layers and perform their operations on the input tensors
        :param inputs:  Input tensor
        :return:        Output tensor
        """
        x = self.max_pool(self.conv1(inputs))        # 1st block
        x = self.max_pool(self.conv2(x))             # 2nd block
        x = self.flatten(x)
        x = self.dense3(self.dense2(self.dense1(x))) # dense layers
        return x

# 3. Preparing the Data

## Creating the Dataset

### Mounting drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Setting ImageDataGenerator

In [ ]:
batch_size = 128
img_height = 256
img_width = 256
rescale = 1./255
data_dir = '/content/drive/My Drive/Datasets/SARS-Cov-2/SARS-Cov-2/'

datagen_aug=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=rescale,
    shear_range=0.05,
    zoom_range=0.05,
    #horizontal_flip = True,
    #brightness_range = [0.8,1.2],
    rotation_range = 15,
    width_shift_range=0.05,
    height_shift_range=0.05
)

datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=rescale
)

train_ds = datagen.flow_from_directory(
    directory = data_dir + 'Train',
    target_size=(img_height, img_width),
    class_mode='binary',
    color_mode="grayscale",
    batch_size=batch_size)

train_ds_aug = datagen_aug.flow_from_directory(
  directory = data_dir + 'Train',
  target_size=(img_height, img_width),
  class_mode='binary',
  color_mode= "grayscale",
  batch_size=batch_size)

valid_ds = datagen.flow_from_directory(
    directory = data_dir + 'Valid',
    target_size=(img_height, img_width),
    class_mode='binary',
    color_mode="grayscale",
    batch_size=batch_size)

Found 1895 images belonging to 2 classes.
Found 1895 images belonging to 2 classes.
Found 451 images belonging to 2 classes.


## Classifying COVID-19 with a Convolutional Neural Network

In [ ]:
# Instantiate and compiling
with tf.device('/device:GPU:0'):
  model = LeNet5()
  model.compile(optimizer = tf.keras.optimizers.Adamax(learning_rate=5e-4),
                loss= 'binary_crossentropy',
                metrics = ['accuracy',
                           tf.keras.metrics.AUC(),
                           tf.keras.metrics.Precision(),
                           tf.keras.metrics.Recall()])

# 4. Setting Callbacks

> Setting Tensorboard logs

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") #directory
file_writer = tf.summary.create_file_writer(log_dir + "/metrics") # for metrics
file_writer.set_as_default()

In [ ]:
callbacks = [
    # Callback to interrupt the training if the validation loss (`val_loss`) stops improving for over 3 epochs:
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
    # Callback to log the graph, losses and metrics into TensorBoard (saving log files in `./logs` directory):
    tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1, write_graph=True),
    tf.keras.callbacks.ModelCheckpoint('best_model', monitor='val_accuracy', verbose=1, save_best_only=True)]

In [ ]:
STEP_SIZE_TRAIN=train_ds.n//train_ds.batch_size
STEP_SIZE_VALID=valid_ds.n//valid_ds.batch_size

model.fit_generator(generator=train_ds,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_ds,
                    validation_steps=STEP_SIZE_VALID,callbacks=callbacks,
                    epochs=10)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
 1/14 [=>............................] - ETA: 0s - loss: 0.6918 - accuracy: 0.4844 - auc: 0.7064 - precision: 0.0000e+00 - recall: 0.0000e+00WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
14/14 [==============================] - ETA: 0s - loss: 0.7391 - accuracy: 0.5252 - auc: 0.5392 - precision: 0.5222 - recall: 0.4328
Epoch 00001: val_accuracy improved from -inf to 0.52604, saving model to best_model
14/14 [==============================] - 17s 1s/step - loss: 0.7391 - accuracy: 0.5252 - auc: 0.5392 - precision: 0.5222 - recall: 0.4328 - val_loss: 0.6665 - val_accuracy: 0.5260 - val_auc: 0.7802 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
14/14 [=======

# 5. Viewing the Results

In [ ]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 909), started 0:59:39 ago. (Use '!kill 909' to kill it.)

<IPython.core.display.Javascript object>

# 6. Implementing Data Augmentation

In [ ]:
# Underfitting
# Instatiate and compiling
with tf.device('/device:GPU:0'):
  model_aug = LeNet5(reg = 2e-5) 
  model_aug.compile(optimizer= tf.keras.optimizers.Adamax(learning_rate=0.001), loss= 'binary_crossentropy', metrics=['accuracy', 
                                                                                                               tf.keras.metrics.AUC(),
                                                                                                               tf.keras.metrics.Precision(),
                                                                                                               tf.keras.metrics.Recall()]) #Strings or methods?
# Setting Tensorboard
!rm -rf ./logs/ 
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") #directory
file_writer = tf.summary.create_file_writer(log_dir + "/metrics") #for metrics
file_writer.set_as_default()

#1000 images - batch_size = 200 1000/200 

callbacks = [
    # Callback to interrupt the training if the validation loss (`val_loss`) stops improving for over 3 epochs:
   # tf.keras.callbacks.EarlyStopping(patience=25, monitor='val_loss'),
    # Callback to log the graph, losses and metrics into TensorBoard (saving log files in `./logs` directory):
    tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1, write_graph=True),
    tf.keras.callbacks.ModelCheckpoint('best_model', monitor='val_accuracy', verbose=1, save_best_only=True)]

STEP_SIZE_TRAIN=train_ds_aug.n//train_ds_aug.batch_size
STEP_SIZE_VALID=valid_ds.n//valid_ds.batch_size

model_aug.fit_generator(generator=train_ds_aug,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_ds, #class_weight=class_weight,
                    validation_steps=STEP_SIZE_VALID,callbacks=callbacks,
                    epochs=100)

Epoch 1/100
14/14 [==============================] - ETA: 0s - loss: 0.7586 - accuracy: 0.5829 - auc_1: 0.5745 - precision_1: 0.6921 - recall_1: 0.2806
Epoch 00001: val_accuracy improved from -inf to 0.59375, saving model to best_model
14/14 [==============================] - 25s 2s/step - loss: 0.7586 - accuracy: 0.5829 - auc_1: 0.5745 - precision_1: 0.6921 - recall_1: 0.2806 - val_loss: 0.7001 - val_accuracy: 0.5938 - val_auc_1: 0.6908 - val_precision_1: 0.7547 - val_recall_1: 0.2186
Epoch 2/100
14/14 [==============================] - ETA: 0s - loss: 0.6854 - accuracy: 0.6484 - auc_1: 0.7293 - precision_1: 0.7529 - recall_1: 0.4348
Epoch 00002: val_accuracy improved from 0.59375 to 0.61979, saving model to best_model
14/14 [==============================] - 23s 2s/step - loss: 0.6854 - accuracy: 0.6484 - auc_1: 0.7293 - precision_1: 0.7529 - recall_1: 0.4348 - val_loss: 0.6758 - val_accuracy: 0.6198 - val_auc_1: 0.6702 - val_precision_1: 0.7500 - val_recall_1: 0.3226
Epoch 3/100
14/

In [ ]:
%tensorboard --logdir logs/fit